# Utils

In [221]:
import os
from typing import Callable, List, Tuple, Union

from librosa.feature.spectral import spectral_flatness
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
from scipy.fft import fft
import scipy.stats
from scipy.stats.mstats import gmean

from utils import Dataset

# Manual feature extraction

## Utilities and preprocessing

In [222]:
def basic_data_cleaning(data: List[pd.DataFrame]) -> List[pd.DataFrame]:
    """
    Assumes DataFrames with "timestamp", "date" and "activity" columns.
    
    Performs cleaning operations:
    - assure format YYYY-MM-DD HH:MM:SS for "timestamp"
    - drop redundant "date" column
    - assure float32 format for "activity"
    
    :param data: list of DataFrames
    :returns: list of cleaned DataFrames
    """
    data = [df.copy() for df in data]  # create copy to avoid side effects
    
    for df in data:
        df["timestamp"] = pd.to_datetime(df["timestamp"], format="%Y-%m-%d %H:%M:%S")
        df.drop("date", axis=1, inplace=True)
        df["activity"] = df["activity"].astype(np.float32)
    
    return data


def get_day_part(df: pd.DataFrame, part: str) -> pd.DataFrame:
    """
    For given DataFrame with "timestamp" column returns only those rows that correspond to the 
    chosen part of day.
    
    Parts are "day" and "night", defined as:
    - "day": [8:00, 21:00)
    - "night": [21:00, 8:00)
    
    :param df: DataFrame to select rows from
    :param part: part of day, either "day" or "night"
    :returns: DataFrame, subset of rows of df
    """
    if part == "day":
        df = df.loc[(df["timestamp"].dt.hour >= 8) & (df["timestamp"].dt.hour < 21)]
    elif part == "night":
        df = df.loc[(df["timestamp"].dt.hour >= 21) | (df["timestamp"].dt.hour < 8)]
    else:
        raise ValueError(f'Part should be "day" or "night", got "{part}"')
        
    return df


def trim_to_length(data: List[pd.DataFrame], length: int) -> List[pd.DataFrame]:
    """
    Trims list of DataFrames to the given length (in terms of row number).
    
    :param data: list of DataFrames
    :param length: target number of rows
    :returns: list of trimmed DataFrames
    """
    data = [df.copy() for df in data]  # create copy to avoid side effects
    data = [df.head(length) for df in data]
    return data


def total_power(df: pd.DataFrame) -> float:
    """
    Calculates total power for given signal. Estimates total power density (i.e. power spectrum) using 
    Welch method and then sums it (integrates discrete signal).
    
    :param df: DataFrame with "activity" column
    :returns: total power
    """
    x = df["activity"].values
    power_spectrum = scipy.signal.welch(x, nperseg=min(len(x), 256))[1]
    return pd.Series(power_spectrum.sum())


def resample_in_domain(df: pd.DataFrame, frequency: str, domain: str) -> pd.DataFrame:
    """
    Resamples time series DataFrame with given frequency, aggregating with a window function.
    Result is either in time or frequency domain, depending on "domain" argument value.
    
    Assumes DataFrame with "timestamp", "date" and "activity" columns. 
    
    Options for "frequency":
    - "hour_quarter": 15 minutes
    - "hour_half": 30 minutes
    - "hour": 60 minutes
    
    Options for "domain":
    - "time": aggregates each period with simple mean (average)
    - "frequency": aggregates each period calculating total power
    
    :param df: DataFrame with columns "datetime" and "activity"
    :param period: "hour_quarter", "hour_half" or "hour"
    :param domain: "time" or "frequency"
    :returns: DataFrame with "timestamp" and "activity" columns
    """
    df = df.copy()  # create copy to avoid side effects
    
    # group with given frequency
    if frequency == "hour_quarter":
        df = df.resample("15min", on="timestamp")
    elif frequency == "hour_half":
        df = df.resample("30min", on="timestamp")
    elif frequency == "hour":
        df = df.resample("H", on="timestamp")
    else:
        raise ValueError(f'Frequency should be "hour_quarter", "hour_half" or "hour", got "{frequency}"')
    
    # aggregate in the proper domain
    if domain == "time":
        df = df.mean()
        df = df.reset_index()
    elif domain == "frequency":
        df = df.agg(total_power)
        
        # clear index, as timestamp index is meaningless in frequency domain
        df = df.reset_index(drop=True)
    else:
        raise ValueError(f'Domain should be "time" or "frequency", got "{domain}"')
    
    return df


def fill_missing_activity(df: pd.DataFrame) -> pd.DataFrame:
    """
    Uses forward filling and then backward filling to fill missing values in "activity" column in DataFrame.
    
    :param data: DataFrame with "activity" column
    :returns: DataFrame with missing values in "activity" column filled
    """
    df = df.copy()  # create copy to avoid side effects
    df["activity"] = df["activity"].ffill().bfill()
    return df


def interdaily_stability(df: pd.DataFrame) -> float:
    """
    Calculates interdaily stability (IS) for single day measurements.
    
    :param df: DataFrame with actigraph measurement for a single day, with "timestamp" and "activity" columns
    :returns: interdaily stability (IS) value
    """
    overall_mean = df["activity"].mean()
    hourly_means = df.groupby(df["timestamp"].dt.hour).mean()
    
    n = len(df["activity"])
    p = len(hourly_means)  # typically 24, for full day measurements
    
    numerator = n * (hourly_means - overall_mean).pow(2).sum()
    denominator = p * (df["activity"] - overall_mean).pow(2).sum()
    
    return float(numerator / denominator)


def intradaily_variability(df: pd.DataFrame) -> float:
    """
    Calculates intradaily variability (IV) for given motor activity measurement.
    
    :param df: DataFrame with actigraph measurement, with "timestamp" and "activity" columns
    :returns: intradaily variability (IV) value
    """
    X = df["activity"]
    n = len(X)
    
    # start from second element of difference, since first one is always NaN
    mean_squared_first_derivative = np.mean(np.square(X.diff().values[1:]))
    population_variance = X.var(ddof=1)  # ddof=1 applies Bessel correction, i.e. division by (N-1) instead of N
    
    return mean_squared_first_derivative / population_variance


def get_m10_l5_and_relative_amplitude(df: pd.DataFrame, time: str = "full_24h") -> Tuple[float, float, float]:
    """
    Calculates features:
    - M10: mean value of 10 most active hours, i.e. with highest sum of activity
    - L5: mean value of 5 least active hours, i.e. with highest sum of activity
    - relative amplitude (RA): defined as (M10 - L5) / (M10 + L5)
    
    Options for "time":
    - "full_24h": default, calculates features for the entire time
    - "day": hours from 8:00 to 21:00
    - "night": hours from 21:00 to 8:00
    
    :param df: DataFrame with "timestamp" and "activity" columns, with raw activity measurements
    :param time: whether to calculate for full 24h, only day or only night
    :returns: M10, L5 and relative amplitude (RA)
    """
    minute_means_across_days = df.groupby([
        df["timestamp"].dt.hour,
        df["timestamp"].dt.minute]).mean()
    minute_means_across_days.index = minute_means_across_days.index.to_flat_index()
    
    # recreate datetime index from tuples (hour, minute); year, month and day are meaningless, but required by Pandas
    index = pd.DataFrame(minute_means_across_days.index.values.tolist(), columns=["hour", "minute"])
    index["year"] = 2000
    index["month"] = 1
    index["day"] = 1
    minute_means_across_days.index = pd.to_datetime(index)

    # we have to pad the array before calculating moving (rolling) sum / averave, since:
    # we have: [mean(00:00 - 01:00), mean(00:01 - 01:01), ..., mean(23:00 - 00:00)]
    # we want: [mean(00:00 - 01:00), ..., mean(23:00 - 00:00), mean(23:01 - 00:01), ..., mean(23:59 - 00:59)]
    # the easiest way to do this is to append copies of the beginning of the activities values to the end
    
    minute_means_across_days = minute_means_across_days.append(minute_means_across_days[:60])
    
    ten_hour_rolls = minute_means_across_days.rolling(window=(10 * 60))
    ten_hour_sums = ten_hour_rolls.sum()
    ten_hour_means = ten_hour_rolls.mean()
    
    five_hour_rolls = minute_means_across_days.rolling(window=(5 * 60))
    five_hour_sums = five_hour_rolls.sum()
    five_hour_means = five_hour_rolls.mean()
    
    # for "night" or "day", select only appropriate hours; remember that Pandas keeps value for 
    # moving (rolling) sum / average at the end of the period, so hours here are shifted
    if time == "day":
        ten_hour_sums = ten_hour_sums[(ten_hour_sums.index.dt.hour >= 9) & (ten_hour_sums.index.dt.hour < 22)]
        ten_hour_means = ten_hour_means[(five_hour_sums.index.dt.hour >= 9) & (ten_hour_means.index.dt.hour < 22)]
        
        five_hour_sums = five_hour_sums[(five_hour_sums.index.dt.hour >= 9) & (five_hour_sums.index.dt.hour < 22)]
        five_hour_means = five_hour_means[(five_hour_means.index.dt.hour >= 9) & (five_hour_means.index.dt.hour < 22)]
    elif time == "night":
        ten_hour_sums = ten_hour_sums[(ten_hour_sums.index.dt.hour >= 22) & (ten_hour_sums.index.dt.hour < 9)]
        ten_hour_means = ten_hour_means[(five_hour_sums.index.dt.hour >= 22) & (ten_hour_means.index.dt.hour < 9)]
        
        five_hour_sums = five_hour_sums[(five_hour_sums.index.dt.hour >= 22) & (five_hour_sums.index.dt.hour < 9)]
        five_hour_means = five_hour_means[(five_hour_means.index.dt.hour >= 22) & (five_hour_means.index.dt.hour < 9)]

    most_active_10_hours_idx = ten_hour_sums["activity"].argmax()
    m10 = ten_hour_means["activity"][most_active_10_hours_idx]
    
    least_active_5_hours_idx = five_hour_sums["activity"].argmin()
    l5 = five_hour_means["activity"][least_active_5_hours_idx]
    
    relative_amplitude = (m10 - l5) / (m10 + l5)
    
    return m10, l5, relative_amplitude


def spectral_flatness(x: np.ndarray) -> float:
    """
    Calculates spectral flatness of a signal, i.e. a geometric mean of the power spectrum divided by 
    the arithmetic mean of the power spectrum.
    
    :param x: 1D Numpy array with signal values
    :returns: spectral flatness value
    """
    power_spectrum = scipy.signal.welch(x, nperseg=min(len(x), 256))[1]
    return gmean(power_spectrum) / power_spectrum.mean()

## Feature extraction

In [223]:
def extract_time_features(df_raw: pd.DataFrame, df_resampled: pd.DataFrame, time: str = "full_24h") -> pd.DataFrame:
    """
    Extracts features from activity signal in time domain.
    
    :param df_raw: DataFrame with "timestamp" and "activity" columns, with raw signal values
    :param df_resampled: DataFrame resampled with another frequency
    :param time: passed to get_m10_l5_and_relative_amplitude(), 
    :returns: DataFrame with a single row representing features
    """
    X = df_resampled["activity"].values
    
    m10, l5, relative_amplitude = get_m10_l5_and_relative_amplitude(df_raw)
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "stddev": np.std(X, ddof=1),  # ddof=1 applies Bessel correction, i.e. division by (N-1) instead of N
        "variance": np.var(X),
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "interdaily_stability": interdaily_stability(df_raw),
        "intradaily_variability": intradaily_variability(df_raw),
        "M10": m10,
        "L5": l5,
        "relative_amplitude": relative_amplitude
    }
    
    return pd.DataFrame([features])

In [224]:
def extract_frequency_features(df_raw: pd.DataFrame, df_frequency: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in frequency domain (precisely, in power domain, since it's assumed 
    that df_frequency has been aggregated with total power using PSD).
    
    :param df_raw: DataFrame with "timestamp" and "activity" columns, with raw signal values
    :param df_frequency: DataFrame in frequency domain
    :returns: DataFrame with a single row representing features
    """
    X = df_frequency.values.ravel()
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "stddev": np.std(X, ddof=1),  # ddof=1 applies Bessel correction, i.e. division by (N-1) instead of N
        "variance": np.var(X),
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "spectral_flatness": spectral_flatness(X)
    }
    
    return pd.DataFrame([features])

# Depresjon

In [225]:
dataset = Dataset(dirpath=os.path.join("data", "depresjon"))
condition = dataset.condition
control = dataset.control

In [220]:
dfs = basic_data_cleaning(condition)
all_dfs = []
for df in dfs:
    df_resampled = resample_in_domain(df, frequency="hour", domain="time")
    df_resampled = fill_missing_activity(df_resampled)
    features = extract_time_features(df, df_resampled, time="full_24h")
    all_dfs.append(features)

pd.concat(all_dfs)

,minimum,maximum,mean,median,stddev,variance,kurtosis,skewness,coeff_of_var,iqr,trimmed_mean,entropy,interdaily_stability,intradaily_variability,M10,L5,relative_amplitude
0,0.000000,1775.400024,146.843735,79.099998,198.571487,39329.011719,16.872655,3.217538,1.350520,214.308335,112.766991,7.600387,0.175629,0.438638,280.083462,7.505000,0.947807
0,0.000000,1184.533325,289.170868,261.433319,271.467957,73490.140625,-0.232590,0.710244,0.937476,457.241667,258.745453,7.788426,0.240762,0.461607,477.964556,16.356000,0.933824
0,0.000000,950.250000,129.167282,50.816666,166.631363,27693.703125,2.877282,1.628733,1.288362,204.808334,98.616768,7.507114,0.092247,0.412314,216.169792,19.506069,0.834467
0,0.000000,1059.316650,151.091476,81.733337,171.847519,29451.755859,1.933566,1.321734,1.135836,252.179164,123.827675,7.621110,0.141852,0.509709,254.190764,14.963556,0.888811
0,0.000000,906.483337,221.277527,178.983337,214.896423,46073.574219,-0.362119,0.730967,0.970038,362.500005,196.210083,7.998547,0.120342,0.535736,332.106759,36.970926,0.799658
0,0.000000,719.416687,75.061485,42.591667,92.996651,8624.487305,9.114864,2.439158,1.237227,102.891665,58.222412,7.591396,0.075246,0.481971,126.034729,14.768222,0.790229
0,0.000000,1099.250000,109.481110,52.458332,142.297028,20192.816406,9.519291,2.511456,1.297954,161.033337,82.409325,7.542342,0.070064,0.544924,156.713153,13.964000,0.836369
0,0.000000,1501.800049,146.327271,5.541667,234.224487,54782.515625,3.348586,1.840380,1.599542,243.775002,96.446457,7.889831,0.135318,0.390030,275.167609,3.369655,0.975805
0,0.000000,1046.016724,85.663643,69.050003,89.986412,8075.061035,34.586817,3.794976,1.049002,118.437495,74.442741,7.817193,0.088310,0.566499,131.412222,13.343111,0.815646
0,0.000000,389.283325,70.900734,64.666664,59.985657,3588.199463,4.015282,1.524170,0.844865,78.016665,63.697311,7.978931,0.023262,0.827363,87.035667,42.232175,0.346594


# Psykose

### Datasets with all features

In [91]:
X_night = pd.merge(
    time_features["X_night"],
    freq_features["X_night"],
    left_index=True,
    right_index=True,
    suffixes=["_time", "_freq"]
)

X_day = pd.merge(
    time_features["X_day"],
    freq_features["X_day"],
    left_index=True,
    right_index=True,
    suffixes=["_time", "_freq"]
)

X_all = pd.merge(
    time_features["X_all"],
    freq_features["X_all"],
    left_index=True,
    right_index=True,
    suffixes=["_time", "_freq"]
)

y_night = time_data["y_night"]
y_day = time_data["y_day"]
y_all = time_data["y_all"]